In [2]:
import os
import json
import pandas as pd
from landy.utils.text_preprocessor import TextPreprocessor
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import TokenTextSplitter
from langchain.vectorstores import Chroma

In [4]:
# data = [
#     """
#     On whether or not this is a good game...
    
#     DFO is a game with a lot of history, thus, it has a lot of quirks and a massive backlog of content for new and old players to enjoy. There are a variety different things to do, but they primarily revolve around PvE content like raids, weekly, and daily dungeons, and less so PvP content. PvP is available, but due to the P2P nature of DFOG, it can make things very difficult, so most players avoid it. It's age means that the game is very well balanced in a lot of areas, and once you catch your stride it can be a game you enjoy for a very long time but also be very overwhelming for people without a guide. The community is also thriving, and there are many people dedicated to helping new and returning players navigate the game. 

#     TLDR: DFO is a very high quality game with a lot of love and money put in to it, so you should come play!
#     """,
#     """
#     On whether or not this game is pay to win (aka P2W)...

#     DFOG is Pay to Progress. There are most things in game you can purchase with in game currency, with varying degrees of time investment, however, paying with Cera (Paid Currency) will vastly hasten this process. Example: a Advanced Clone avatar set will typically be around 6 million gold. This takes about 2-3 hours to earn depending on how you do it, and you must purchase it from another player or buy it via the Event shop for the same price. You can also pay 10$ for the item by itself, or 30-45$ for this item to come in a package with other items that may be of high value, and Advanced Clones are seen as mandatory to enter most content as a baseline. You'll find similar instances of this happening with other items such as Auras, Pets, and Titles. These items give direct damage benefits and can be attained in game only via trading if you're fully free to play. You also have a monthly paid subscription-like service called Neo: Premium which provides more Fatigue Points to let you play for longer. 

#     TLDR: DFOG leans more towards Pay to Play, with options for most players to attain items via in game trading. 
#     """,
#     """
#     On farming gold...

#     You can make gold by going and farming Canyon Hills on the highest difficulty you can do comfortably (1:30s-2:00m) and selling all Uncommon equipment, and dissing all Common-Unique equipment. Doing this allows you to then either sell the Clear Cube Fragments on the Auction Hall, or hoard them for later use. You also gain Beryl, Golden Beryl, Ryan Cores, Harmonious Crystals, and a chance at Legendary cards from the purple card flip at the end of the dungeon, as well as Ustable Rifts for additional materials. You can also sell Golden Beryl you obtain from doing Advanced Dungeons for a fair price. Be sure to keep any Dark Fragmented Abyss pieces! They are labeled as Unique grade and very useful for minmaxing at the top end. You can also use Advanced Dungeon materials every week to buy 4x Card Albums sold by Meryl Pioneer and Meister Curio and sell the resulting cards. 

#     TLDR: Farm Canyon Hills, sell Uncommon equipment, diss all else, sell clear cubes to the Auction Hall, or sell Golden Beryl. Or $$$ and merch.
#     """,
#     """
#     On getting or increasing your fame (aka fp, fame points)...

#     Every item you have, has fame on it. Thus, upgrading those items either by getting stronger ones (eg: going from 30 elemental enchant to 35) or improving your armor/accs/wep/sub equipments via fusing will increase your fame. The best way to do this is to simply upgrade your items via fusing, getting better enchants, emblems, runes, and talismans, or by reinforcing/amping. There is no 1 stop shop to the exact rout, as there are many variables to increasing your fame (aka, getting stronger) and it will vary depending on your current progression in the game. It's much easier to get fame as someone who is 19k fame, versus someone who is 43k fame. The higher fame you are, the more difficult it will be to obtain. 
#     """
# ]
# # Make new df
# new = pd.DataFrame(data, columns=['blog'])
# # Add a date col
# new['date'] = '2023-05-03'
# # Load old
# old_fp = 'data/interim/blogs.json'
# old = pd.read_json(old_fp)
# # old old manually inserted
# old = old[old.date != '2023-05-03']
# # Manually insert new
# df = pd.concat([old,new]).reset_index(drop=True)
# # Save
# df.to_json(old_fp)

In [3]:
# Get the absolute path of the directory containing the script
script_dir = os.path.dirname(os.path.abspath('__main__'))
# Construct a file path relative to the script's directory
data_file = os.path.join(script_dir, 'data', 'interim',
                         'blogs.json')
# Load the data from a json file
with open(data_file, 'r') as f:
    data = json.load(f)
# Grab just the blog posts data in a 2d array
posts = [post for post in data['blog'].values()]

preprocessor = TextPreprocessor()
text_splitter = TokenTextSplitter(chunk_size=6500)  
embedder = OpenAIEmbeddings()

processed_texts = [preprocessor.preprocess(text) for text in posts]
docs = text_splitter.create_documents(processed_texts)
docs = text_splitter.split_documents(docs)

persist_dir = os.path.join(script_dir, 'db')
db = Chroma.from_documents(documents=docs,
                           embedding=embedder,
                           persist_directory=persist_dir)
db.persist()

/home/jovyan/workspace/landy/landy/utils/text_preprocessor.py:73: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")
Using embedded DuckDB with persistence: data will be stored in: /home/jovyan/workspace/landy/db
